In [32]:
using LowLevelFEM
gmsh.initialize()

In [33]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000546398s, CPU 0.000403s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00998077s, CPU 0.009974s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [34]:
mat = Material("body")
problem = Problem([mat], type=:PlaneStress)

Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [35]:
supp = displacementConstraint("left", ux=0, uy=0)
load0 = load("right", fx=1)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, 1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [36]:
u0 = applyBoundaryConditions(problem, [BoundaryCondition("body", ux=0, uy=0)])

VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [37]:
applyBoundaryConditions!(u0, [BoundaryCondition("left", ux=0, uy=0)])

In [38]:
v0 = applyBoundaryConditions(problem, [BoundaryCondition("body", ux=0, uy=0)])

VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [39]:
applyBoundaryConditions!(v0, [BoundaryCondition("body", ux=0, uy=0)])

In [40]:
K = stiffnessMatrix(problem)
M = massMatrix(problem)
f = loadVector(problem, [load0])

VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [41]:
#FEM.applyBoundaryConditions!(K, M, f, [supp])

In [42]:
@time Tmin = smallestPeriodTime(K, M, [supp])

  0.002648 seconds (1.81 k allocations: 1.312 MiB)


4.7358837432307966e-8

In [43]:
u, v = CDM(K, M, f, [supp], u0, v0, 100, Tmin / π)

(VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0579472511892285e-6 8.523773933951735e-7; 0.0 0.0 … -2.200321628761794e-7 -2.669881057286106e-7], [0.0, 1.5074786152874593e-8, 3.0149572305749186e-8, 4.5224358458623776e-8, 6.029914461149837e-8, 7.537393076437297e-8, 9.044871691724757e-8, 1.0552350307012216e-7, 1.2059828922299674e-7, 1.3567307537587133e-7  …  1.3567307537587117e-6, 1.3718055399115862e-6, 1.3868803260644607e-6, 1.4019551122173352e-6, 1.4170298983702097e-6, 1.4321046845230842e-6, 1.4471794706759587e-6, 1.4622542568288332e-6, 1.4773290429817077e-6, 1.4924038291345822e-6], Int64[], 100, :v2D, Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))), VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 

In [44]:
showDoFResults(u, :uvec, visible=true)

11

In [45]:
showDoFResults(v, :vvec)

12

In [46]:
e1 = solveStrain(u)

TensorField
[[0.0 0.0 … 1.8590123454898325e-6 2.0213449967894e-6; 0.0 0.0 … -2.113242821705345e-7 1.0167228737322037e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -3.8049124311737865e-7 -5.350122150267052e-7], [0.0 0.0 … 2.6134530399467953e-6 2.4998547733895536e-6; 0.0 0.0 … -6.095838570387613e-7 8.36861099111121e-8; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -4.736710227141284e-7 -2.8781854433472847e-7], [0.0 0.0 … 2.685912688594314e-6 2.533418488049503e-6; 0.0 0.0 … -4.388123400554417e-8 -2.1457696300200853e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -3.1400060877591433e-7 -2.8398925251628593e-7], [0.0 0.0 … 1.6877211904049914e-6 1.5327790733494963e-7; 0.0 0.0 … 4.299991992432328e-7 6.625068771048187e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -6.119842268743806e-7 -2.913666720086649e-7], [0.0 0.0 … 2.7827386820207413e-6 2.7947478476554605e-6; 0.0 0.0 … 1.8767051300657863e-7 1.3682919899344348e-7; … ; 0.0 0.0 … 0.0 0.0; -0.0 -0.0 … -5.715207228085076e-7 -5.862740543374403e-7], [0.0 0.0 … 1.0523913107031935e-

In [47]:
showStrainResults(e1, :e)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


13

In [48]:
e2 = solveStrain(u, DoFResults=true)

TensorField
[0.0 0.0 … 1.1431487640867218e-6 9.240295927221739e-7; 0.0 0.0 … 1.0520085103202502e-6 6.567723986208344e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -6.324411986999221e-7 -2.161622250961719e-7]

In [49]:
showDoFResults(e2, :e)

14

In [50]:
e3 = elementsToNodes(e1)

TensorField
[0.0 0.0 … 1.1431487640867218e-6 9.240295927221739e-7; 0.0 0.0 … 5.260042551601251e-7 3.283861993104172e-7; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … -6.324411986999221e-7 -2.161622250961719e-7]

In [51]:
showDoFResults(e3, :e)

15

In [52]:
s1 = solveStress(u)

TensorField
[[0.0 0.0 … 0.3445390136318786 0.3932857248947854; 0.0 0.0 … -0.03251142802623608 0.015641890365110826; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.4749421634862959 0.42887281811142797; 0.0 0.0 … -0.09378213185211713 0.012874786140171093; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.46152512267234286 0.43344734470242907; 0.0 0.0 … -0.006750959077776026 -0.033011840461847464; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.3538008334275957 0.06840685836055607; 0.0 0.0 … 0.06615372296049735 0.10192413493920288; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.516039908435269 0.5201572156912155; 0.0 0.0 … 0.02887238661639671 0.021050645998991306; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.18989174282535043 0.09015291908362064; 0.0 0.0 … -0.01187694217114177 0.0403636902931631; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.3653429678701632 … 0.9461325434150245 0.8220185024937752; 0.0 -0.009281002657058966 … -0.0032418302449

In [53]:
showStressResults(s1, :s)

16

In [54]:
s2 = solveStress(u, DoFResults=true)

TensorField
[0.0 0.0 … 0.24868132605881654 0.22881412762791714; 0.0 0.0 … 0.08092373156309618 0.050520953740064184; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

In [55]:
showDoFResults(s2, :s)

17

In [56]:
err1 = fieldError(e1)

TensorField
[0.0 0.0 … 5.619382564218145e-8 2.6687541742521416e-7; 0.0 0.0 … 8.682361955913087e-9 6.168677126214177e-8; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 2.214045091842815e-7 1.4897442100820287e-7]

In [57]:
showDoFResults(err1, :tensor)

18

In [58]:
u2, v2 = HHT(K, M, f, [supp], u0, v0, 100, Tmin / π)

(VectorField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 1.3933835462338468e-9 … 1.1345722688456155e-6 8.207752802239212e-7; 0.0 -1.3326588533142092e-10 … -1.064447856926893e-7 -1.818974963371531e-7], [0.0, 1.5074786152874593e-8, 3.0149572305749186e-8, 4.5224358458623776e-8, 6.029914461149837e-8, 7.537393076437297e-8, 9.044871691724757e-8, 1.0552350307012216e-7, 1.2059828922299674e-7, 1.3567307537587133e-7  …  1.3567307537587117e-6, 1.3718055399115862e-6, 1.3868803260644607e-6, 1.4019551122173352e-6, 1.4170298983702097e-6, 1.4321046845230842e-6, 1.4471794706759587e-6, 1.4622542568288332e-6, 1.4773290429817077e-6, 1.4924038291345822e-6], Int64[], 100, :v2D, Problem("body1-2D", :PlaneStress, 2, 2, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))), VectorField(Matrix{Float64}[], [

In [59]:
showDoFResults(u2, :uvec)

19

In [60]:
showDoFResults(v2, :vvec)

20

In [61]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [62]:
gmsh.finalize()